In [1]:
!git clone https://github.com/jan1na/Neural-Cellular-Automata.git

import os

%cd Neural-Cellular-Automata


Cloning into 'Neural-Cellular-Automata'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 18 (delta 7), reused 8 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (18/18), 9.41 KiB | 9.41 MiB/s, done.
Resolving deltas: 100% (7/7), done.
/content/Neural-Cellular-Automata


In [2]:
!pip install -q medmnist

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
from medmnist import PathMNIST
import medmnist
from tqdm import tqdm
from train_utils import train, evaluate

from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 794.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 26.9 MB/s eta 0:00:00
Mounted at /content/drive


In [3]:
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = PathMNIST(split='train', transform=transform, download=True)
val_dataset = PathMNIST(split='val', transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)


100%|██████████| 206M/206M [01:41<00:00, 2.02MB/s]


In [4]:
class CNNBaseline(nn.Module):
    def __init__(self, num_classes=9):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU()
        )
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNNBaseline().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

save_path = '/content/drive/MyDrive/NCA/best_cnn_pathmnist.pth'
best_acc = 0

for epoch in range(1, 31):
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)

    print(f"Epoch {epoch:02d}: Train Acc = {train_acc:.4f}, Val Acc = {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), save_path)
        print(f"Best model saved to: {save_path}")

    if val_acc > 0.90:
        print("Converged with >90% accuracy!")
        break

print("Training complete.")

Epoch 01: Train Acc = 0.5577, Val Acc = 0.6647
Best model saved to: /content/drive/MyDrive/NCA/best_cnn_pathmnist.pth
Epoch 02: Train Acc = 0.6954, Val Acc = 0.7408
Best model saved to: /content/drive/MyDrive/NCA/best_cnn_pathmnist.pth
Epoch 03: Train Acc = 0.7357, Val Acc = 0.7486
Best model saved to: /content/drive/MyDrive/NCA/best_cnn_pathmnist.pth
Epoch 04: Train Acc = 0.7650, Val Acc = 0.7648
Best model saved to: /content/drive/MyDrive/NCA/best_cnn_pathmnist.pth
Epoch 05: Train Acc = 0.7878, Val Acc = 0.8146
Best model saved to: /content/drive/MyDrive/NCA/best_cnn_pathmnist.pth
Epoch 06: Train Acc = 0.8035, Val Acc = 0.8305
Best model saved to: /content/drive/MyDrive/NCA/best_cnn_pathmnist.pth
Epoch 07: Train Acc = 0.8156, Val Acc = 0.8138
Epoch 08: Train Acc = 0.8273, Val Acc = 0.8345
Best model saved to: /content/drive/MyDrive/NCA/best_cnn_pathmnist.pth
Epoch 09: Train Acc = 0.8373, Val Acc = 0.8603
Best model saved to: /content/drive/MyDrive/NCA/best_cnn_pathmnist.pth
Epoch 10: